## Build a Quadratic Program from scratch and transform to qubo and ising model

In [1]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
import dimod

In [2]:
qp = QuadraticProgram("Example")
n = 3
for j in range(n):
            qp.binary_var('x' + "_" + str(j))
linear_objective = np.array([-1,0,-1])
Q = np.array([[-2,0,1],
              [0,2,-2],
              [1,-2,0]])
quadratic_objective = {}
for i in range(n):
    for j in range(i, n):
        var_i = 'x' + "_" + str(i)
        if i == j:
            quadratic_objective[var_i, var_i] = Q[i, i]
        else:
            var_j = 'x' + "_" + str(j)
            quadratic_objective[var_i, var_j] = Q[i, j] + Q[j, i]
qp.linear_constraint(linear=np.array([0,1,-1]), sense='==', rhs=0,
                        name='linear_cstr')
qp.minimize(linear=linear_objective, quadratic=quadratic_objective)
print(qp)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Example

Minimize
 obj: - x_0 - x_2 + [ - 4 x_0^2 + 4 x_0*x_2 + 4 x_1^2 - 8 x_1*x_2 ]/2
Subject To
 linear_cstr: x_1 - x_2 = 0

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1

Binaries
 x_0 x_1 x_2
End



In [3]:
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)
print(f"Auto calculated {conv.penalty=}")
print(qubo)

Auto calculated conv.penalty=13.0
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Example

Minimize
 obj: - x_0 - x_2 + [ - 4 x_0^2 + 4 x_0*x_2 + 30 x_1^2 - 60 x_1*x_2 + 26 x_2^2
      ]/2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1

Binaries
 x_0 x_1 x_2
End



In [4]:
bqm_binary = dimod.as_bqm(qubo.objective.linear.to_array(), qubo.objective.quadratic.to_array(), dimod.BINARY)
print(bqm_binary)
bqm_ising = bqm_binary.to_ising()
print("Ising Model form coefficients:")
print(bqm_ising)

AdjVectorBQM({0: -3.0, 1: 15.0, 2: 12.0}, {(0, 2): 2.0, (1, 2): -30.0}, 0.0, 'BINARY')
Ising Model form coefficients:
({0: -1.0, 1: 0.0, 2: -1.0}, {(0, 2): 0.5, (1, 2): -7.5}, 5.0)
